# Recurrent Neural Networks

Here we'll play with a hand-rolled RNN that works on text data and see what we can do with it.

This comes from Andrej Karpathy (currently director of AI at Tesla, and my old labmate who I'm coauthor on a paper with)

In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np


class BasicRNN:
    def __init__(self, data, hidden_size=128, seq_length=64,learning_rate=1e-1):
        self.data = data;
        self.chars = list(set(data))
        self.data_size, self.vocab_size = len(data), len(self.chars)
        print ('data has %d characters, %d unique.' % (self.data_size, self.vocab_size))
        
        # what is our "alphabet" ?  We're going to number the characters from 
        # 0 to numUniqueCharacters
        # and store forward and reverse mappings 
        # (what index is this char, what char is at this index?)
        self.char_to_ix = { ch:i for i,ch in enumerate(self.chars) }
        self.ix_to_char = { i:ch for i,ch in enumerate(self.chars) }

        # hyperparameters
        self.hidden_size = hidden_size # size of hidden layer of neurons
        self.seq_length = seq_length # number of steps to unroll the RNN for
        self.learning_rate = learning_rate #for optimization

        # model parameters
        # these are the weights that we're training
        self.Wxh = np.random.randn(self.hidden_size, self.vocab_size)*0.01 # input to hidden
        self.Whh = np.random.randn(self.hidden_size, self.hidden_size)*0.01 # hidden to hidden
        self.Why = np.random.randn(self.vocab_size, self.hidden_size)*0.01 # hidden to output
        self.bh = np.zeros((self.hidden_size, 1)) # hidden bias
        self.by = np.zeros((self.vocab_size, 1)) # output bias

    # measures the error (how far arr of ar the targets from what the model predicts)
    # and the gradients (the calculus used to improve the weights)
    def lossFun(self,inputs, targets, hprev):
      """
      inputs,targets are both list of integers.
      hprev is Hx1 array of initial hidden state
      returns the loss, gradients on model parameters, and last hidden state
      """
      xs, hs, ys, ps = {}, {}, {}, {}
      hs[-1] = np.copy(hprev)
      loss = 0
      # forward pass.  This computes the actual error
      for t in range(len(inputs)):
        xs[t] = np.zeros((self.vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(self.Wxh, xs[t]) + np.dot(self.Whh, hs[t-1]) + self.bh) # hidden state
        ys[t] = np.dot(self.Why, hs[t]) + self.by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars (apply softmax)
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
      
    
      # backward pass: compute gradients going backwards
      # this is the calculus part
      dWxh, dWhh, dWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
      dbh, dby = np.zeros_like(self.bh), np.zeros_like(self.by)
      dhnext = np.zeros_like(hs[0])
      for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(self.Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(self.Whh.T, dhraw)
      for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
      return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

    def sample(self, h, seed_ix, n):
      """ 
      sample a sequence of integers from the model 
      h is memory state, seed_ix is seed letter for first time step
      """
      x = np.zeros((self.vocab_size, 1))
      x[seed_ix] = 1
      ixes = []
      for t in range(n):
        #set up the hidden state properly
        h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)
        y = np.dot(self.Why, h) + self.by
        p = np.exp(y) / np.sum(np.exp(y)) #run the input through the network
        #pick the next character based on the probabilities output by the network
        ix = np.random.choice(range(self.vocab_size), p=p.ravel()) 
        x = np.zeros((self.vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
      return ixes

    # perform the optimization to find the best weights
    def train(self, nIters = 10000):
        n, p = 0, 0
        mWxh, mWhh, mWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
        mbh, mby = np.zeros_like(self.bh), np.zeros_like(self.by) # memory variables for Adagrad
        smooth_loss = -np.log(1.0/self.vocab_size)*self.seq_length # loss at iteration 0
        while n < nIters:
          # prepare inputs (we're sweeping from left to right in steps seq_length long)
          if p+self.seq_length+1 >= len(self.data) or n == 0: 
            hprev = np.zeros((self.hidden_size,1)) # reset RNN memory
            p = 0 # go from start of data
          inputs = [self.char_to_ix[ch] for ch in self.data[p:p+self.seq_length]]
          targets = [self.char_to_ix[ch] for ch in self.data[p+1:p+self.seq_length+1]]

          # sample from the model now and then
          if n % 200 == 0:
            sample_ix = self.sample(hprev, inputs[0], 200)
            txt = ''.join(self.ix_to_char[ix] for ix in sample_ix)
            print( '----\n %s \n----' % (txt, ))
          #print("inp: ", inputs, "targets: ", targets, "hprev: ", hprev)
          # forward seq_length characters through the net and fetch gradient
          loss, dWxh, dWhh, dWhy, dbh, dby, hprev = self.lossFun(inputs, targets, hprev)
          smooth_loss = smooth_loss * 0.999 + loss * 0.001
          if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss) )# print progress
  
          # perform parameter update with Adagrad
          for param, dparam, mem in zip([self.Wxh, self.Whh, self.Why, self.bh, self.by], 
                                        [dWxh, dWhh, dWhy, dbh, dby], 
                                        [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam
            param += -self.learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

          p += self.seq_length # move data pointer
          n += 1 # iteration counter 

In [2]:
x = 1.0/7.0
s = str(x)

pattern = s[2:8]
pattern *= 1000

print( s )
print( pattern[:30] )

0.14285714285714285
142857142857142857142857142857


In [3]:
repeatingRNN = BasicRNN( pattern )

data has 6000 characters, 6 unique.


In [4]:
repeatingRNN.train()

----
 14422255557844475781228778878411747227527854578578741185427475111724888147278857448885424287548741878424447874752748577744741774458844774287851147455147417451777725555814825815455224222575258577247781 
----
iter 0, loss: 114.672614
iter 100, loss: 115.831142
----
 81714285714285714285714281428571428571428571428571428571428571428571428571428571428571428571428572428571428571428571428571422571428571428571428571428571428571428571428571428571428571428571428571428571 
----
iter 200, loss: 107.154893
iter 300, loss: 97.082856
----
 71428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428542857742857142857142857142857142857142857142857142857172854142857142857142857142857142 
----
iter 400, loss: 87.910477
iter 500, loss: 79.589859
----
 428571428571478571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571

In [5]:
def fizzbuzz( N ):
    out = ""
    for i in range( 1, N+1 ):
        if i % 3 == 0:
            out += "fizz"
            if i % 5 == 0:
                out += "buzz"
        elif i % 5 == 0:
            out += "buzz"
        else:
            out += str( i )
        out += " "
    return out

pattern = fizzbuzz( 100 )
display( pattern )

'1 2 fizz 4 buzz fizz 7 8 fizz buzz 11 fizz 13 14 fizzbuzz 16 17 fizz 19 buzz fizz 22 23 fizz buzz 26 fizz 28 29 fizzbuzz 31 32 fizz 34 buzz fizz 37 38 fizz buzz 41 fizz 43 44 fizzbuzz 46 47 fizz 49 buzz fizz 52 53 fizz buzz 56 fizz 58 59 fizzbuzz 61 62 fizz 64 buzz fizz 67 68 fizz buzz 71 fizz 73 74 fizzbuzz 76 77 fizz 79 buzz fizz 82 83 fizz buzz 86 fizz 88 89 fizzbuzz 91 92 fizz 94 buzz fizz 97 98 fizz buzz '

In [6]:
fbRNN = BasicRNN( fizzbuzz(100) )
fbRNN.train()

data has 413 characters, 15 unique.
----
  78b72zz5487u2u74b b2bi1zbzf19b2i58583b56u5363  1uf538z9138 2i4zuz2f58iu66zuf8 67494zzib4555419i93i8u5 64934f28fi89 z7734z925589u143f4 8 11481zubiuz2432661 b74u4zf7431fb269z367b844 57u75597f244u41z4bf 
----
iter 0, loss: 173.315227
iter 100, loss: 176.917809
----
 z7z  zzz   z z6zzzz b 4zib3 8zz2zfz 2 izzzz7 2 8 zzz i3 z8   z3 zz5 zbz8zz   1 i zzzuzzz z7z zz252z2z3iz 8 zzf  zz  zz6zzz zzb2fz2  zz 72 3  z8z fz8zzz2z z9z8 2 9  zzzz i    1izzz3718  8 8    2 zzz  z 
----
iter 200, loss: 174.981856
iter 300, loss: 171.559569
----
  b1zz3z bbfz iz uu 5 145i u 31 54zz8 4 b9ziz  4z f5 33zf5 fz z86z 1z b2zzzii61 7zb5f 3z6zz 39i3 f68zz 8z2z i2 8 f6 f f4 3z 4 uz fiz bff uz b u4 65 fzz 2zz1i 562z fb3z7 buzz6zzz 6 iizizzz1 u 68zzb4 bu1 
----
iter 400, loss: 167.558400
iter 500, loss: 163.472130
----
  2b2uz 8zu 76ziz fz1zz bfzzb892zzz 8zzz 5fzz uzzzi izzzizzzz izzzzizzi f 16zziz 52zzz 3b7 6bbuzzzzz fizzii fizi 7zzzzzzzzzzzz bfzz9 4zzizzzz 7fz 36 88izzzzzzzz

In [7]:
import urllib
aiw = urllib.request.urlopen( "http://www.gutenberg.org/cache/epub/19033/pg19033.txt" )
#dir(aiw)
aiw = str( aiw.read(), 'utf=8' )
aiw[ :100 ]

'\ufeffThe Project Gutenberg EBook of Alice in Wonderland, by Lewis Carroll\r\n\r\nThis eBook is for the use o'

In [8]:
aiwRNN = BasicRNN( aiw, hidden_size=512,seq_length=256 )

data has 74700 characters, 87 unique.


In [9]:
aiwRNN.train( 20000 )

----
﻿Oùof$P04UPdr[E6!qa!F@e G2?;﻿#)_zzO Gt#d66M!!
*!hk
-88.d'5*TSHU.G3H::k6
ùD&4"_x*f?X; t
----
iter 0, loss: 1143.272370
iter 100, loss: 1551.310539
----
 eG
G8GWGWGWGmGWGmGeGmGeGmGDGWFtGmGmhgKWKeGWGDGWGQGmGmGWGWGWGWGeGmGmGWGDGeGmGWGmGWFWGmKHGtGlGmGWGeGQGDGWKWGfGeGmGLGWFWGeGWGfGWGWGmGrGmGmGWGmGWGWGQGtGWGWGDGDGmGWGmGTGmGWGWGmGmGWGQGWGWFmGWG 
----
iter 200, loss: 1649.651076
iter 300, loss: 1738.955289
----
 rtIenJAJ Jrt:alt$JdJ,JrJuJre,J's.JrJocrJoauJrtItrJrJncrtrJrthtreltuJuJrJmJosrtrJra,JNJlJoaoJrJuJ'JwtuJoJoJrtwmhJ,JNJ-JrtoJrereotPaoJmJoarJrJnJomutpJrJrJwJ6tnJ/wdloJoerJNJnyA db,JAtotrs)JrwrsuJrautrt'e 
----
iter 400, loss: 1707.367324
iter 500, loss: 1654.247462
----
 i3y#u#hn
t a yet#dnh#hjk##b#y
"eE heSro#rqm scU unu ofats#, i
 #iniEananr#i heanaetck6scdcw oow#g ili
inirg#gnl an, h tnoie#mcoeh v T d bAz,m#jAX i hlasol facu unyeu
i awi h
d i h  
----
iter 600, loss: 1616.504937
iter 700, loss: 1558.239825
----
/r/E/d/G/1/ / /h/3/l/E/ /*/ /D/d/Oo /t/s/z/ // / /O/w/1/G/3/*/]/0/ /D/ 

KeyboardInterrupt: 

In [ ]:
aiwRNN.train( 1000 )